In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_bnd3,wp_bnd7,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,0.029934,0.112914,0.335343,0.354057,6052.592985,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,0.029932,0.112904,0.335286,0.354033,6178.213349,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,0.029930,0.112893,0.335229,0.354008,6313.892022,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,0.029928,0.112882,0.335173,0.353984,6459.629005,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,0.029926,0.112872,0.335116,0.353960,6615.424298,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1027
Data columns (total 28 columns):
PET             1028 non-null float64
VPD             1028 non-null float64
air_temp        1028 non-null float64
doy             1028 non-null int64
precip          1028 non-null float64
soil_temp       1028 non-null float64
sw_in           1028 non-null float64
wind_speed      1028 non-null float64
year            1028 non-null int64
wp_RNET         1028 non-null float64
wp_ch4_gf       1028 non-null float64
wp_co2_gf       1028 non-null float64
wp_er           1028 non-null float64
wp_gpp          1028 non-null float64
wp_h            1028 non-null float64
wp_le           1028 non-null float64
wp_bnd1         1028 non-null float64
wp_bnd2         1028 non-null float64
wp_bnd3         1028 non-null float64
wp_bnd7         1028 non-null float64
wp_evi          1028 non-null float64
wp_lswi2        1028 non-null float64
wp_ndvi         1028 non-null float64
wp_LST.day      1028 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_le"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1028, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.974221571607
CV Scores:  0.979055419168, 0.972024862707, 0.977326484541, 0.976257456389, 0.975560173786, 0.975834280973, 0.967662947713, 0.9710407782, 0.978618611034, 0.968834701558
OOB score: 0.976372714161
Feature Importances:
('wp_LST.night', 0.30050351496455008)
('air_temp', 0.17136197811698284)
('PET', 0.11279740808089386)
('wp_LST.day', 0.1075665409929317)
('VPD', 0.099355881590416098)
('wp_ndvi', 0.059172615069811786)
('sw_in', 0.049893407074553184)
('wp_evi', 0.033858317817732436)
('wp_lswi2', 0.028894331350708482)
('LW_IN.wp', 0.026011176593612885)
('wind_speed', 0.0092240498517887556)
('precip', 0.0013607784960178991)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.974298277648
CV Scores:  0.980242310546, 0.972391799308, 0.97940560985, 0.973555375576, 0.969264083394, 0.978106505828, 0.967943019163, 0.967547176642, 0.978083466745, 0.976443429426
Feature Importances:
('wp_evi', 0.13073665247531602)
('wp_LST.night', 0.12940294270054836)
('wp_lswi2', 0.12001768321012024)
('air_temp', 0.1008874562818345)
('sw_in', 0.087274534053301767)
('wp_LST.day', 0.083123163446570184)
('PET', 0.080671352481080622)
('wp_ndvi', 0.07333951417393636)
('LW_IN.wp', 0.066866617943865253)
('wind_speed', 0.058639985347717721)
('VPD', 0.053640581520343129)
('precip', 0.01539951636536583)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.935421269581
CV Scores:  0.952718596603, 0.936706659888, 0.914005511526, 0.942301918074, 0.939620052722, 0.944515477592, 0.952418770112, 0.875949914371, 0.963022804848, 0.932952990075


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 74.80708
Step #501, epoch #50, avg. loss: 1.97430
Step #1001, epoch #100, avg. loss: 0.63195
Step #1501, epoch #150, avg. loss: 0.38991
Step #2001, epoch #200, avg. loss: 0.25858
Step #2501, epoch #250, avg. loss: 0.24105
Step #3001, epoch #300, avg. loss: 0.16806
Step #3501, epoch #350, avg. loss: 0.14844
Step #4001, epoch #400, avg. loss: 0.13162
Step #4501, epoch #450, avg. loss: 0.11849
Step #1, avg. loss: 48.07025
Step #501, epoch #50, avg. loss: 1.92813
Step #1001, epoch #100, avg. loss: 0.60303
Step #1501, epoch #150, avg. loss: 0.37988
Step #2001, epoch #200, avg. loss: 0.29934
Step #2501, epoch #250, avg. loss: 0.23418
Step #3001, epoch #300, avg. loss: 0.21616
Step #3501, epoch #350, avg. loss: 0.16155
Step #4001, epoch #400, avg. loss: 0.13750
Step #4501, epoch #450, avg. loss: 0.12795
Step #1, avg. loss: 62.38238
Step #501, epoch #50, avg. loss: 1.98289
Step #1001, epoch #100, avg. loss: 0.67321
Step #1501, epoc

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x114f2c410>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)